# Question answering con sintesi del contesto

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain_openai import ChatOpenAI
from bs4 import BeautifulSoup as Soup

import os


In [6]:
# Definisce l'URL di partenza per il crawling
url = "https://www.develhope.co/"

# Crea un loader ricorsivo che estrae il testo dalle pagine HTML fino a una profondità di 2 link
loader = RecursiveUrlLoader(
    url=url,
    max_depth=2,
    extractor=lambda x: Soup(x, "html.parser").text.replace("\n", " ").replace("\t", " ").strip()
)

# Scarica e carica i documenti estratti dal sito web
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=40, chunk_overlap=100)
all_splits = text_splitter.split_documents(docs)

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=OpenAIEmbeddings(),
    persist_directory="develhope/rag_lessons"
)  

In [14]:
question = "Sono un HR manager di un'azienda che cerca attivamente del personale. Può Develhope aiutarmi?"
docs = vectorstore.similarity_search(question, k=4)
print(len(docs))

4


In [7]:
for i in docs:
    print(i.page_content)
    print("-----")

Develhope: diventa Developer, Data Analyst o Marketing Specialist                     HomeCorsiChi SiamoRecensioniAziendeFAQBlogAccediRegistrati OraSelect LanguageItalianoItalianoIscrivitiHomeChi SiamoCorsiCorso Live AI-Powered4 o 10 settimaneCorso Flex DataFino a 12 mesiCorso Flex Back-EndFino a 12 mesiCorso Flex WebFino a 12 mesiCorso Live Web5 mesiCorso Live Digital Marketing/AI10 settimaneCorso Live Data & AI 5 mesiCorso Live UX/UI 10 settimaneIscrivitiChi SiamoRecensioniAziendeFAQBlogIscrivitiHomeChi SiamoCorsiCorso Live AI-Powered4 o 10 settimaneCorso Flex DataFino a 12 mesiCorso Flex Back-EndFino a 12 mesiCorso Flex WebFino a 12 mesiCorso Live Web5 mesiCorso Live Digital Marketing/AI10 settimaneCorso Live Data & AI 5 mesiCorso Live UX/UI 10 settimaneIscrivitiChi SiamoRecensioniAziendeFAQBlogCostruiamo insieme la tua Carriera Tech Corsi di programmazione, data tech e marketing per programmare un futuro migliore. Il tuo.Paghi se trovi lavoro.Scopri i CorsiScopri i CorsiCostruiamo 

In [9]:
# Creiamo una chain per riformulare i documenti analizzati

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate


small_llm = ChatOpenAI(model="gpt-4o-mini",
                       api_key=os.getenv("openai_api_key"))

# Prompt
prompt = PromptTemplate.from_template(
    "Sintetizza il contenuto di questi documenti senza tralasciare dettagli: {docs}"
)


# Chain
def format_docs(docs):
    return "\n\n".join(doc.metadata["description"] + " " + doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | small_llm | StrOutputParser()

# Run
docs = vectorstore.similarity_search(question, k=7)

knowledge_base = chain.invoke(docs)

knowledge_base

"Develhope offre una serie di corsi online dedicati a chi desidera intraprendere una carriera nel settore tecnologico. I programmi principali includono:\n\n1. **Sviluppatore Back-end Java**: Un corso per apprendere le competenze necessarie per lavorare come sviluppatore back-end nelle migliori aziende tech. Il corso è 100% online, pratico, con supporto tutor e certificato finale, oltre a un aiuto nella ricerca di lavoro.\n\n2. **Sviluppatore Full-Stack**: Simile al corso di back-end, questo programma forma gli studenti ad affrontare sfide di sviluppo sia sul lato front-end che back-end, con le stesse modalità di studio e supporto.\n\n3. **Percorsi per sviluppatori con basi**: Per chi ha conoscenze di base in Frontend, Backend o Data e desidera migliorarsi, Develhope offre corsi pratici, con una durata di 4 settimane (Corso Breve) o 10 settimane (Corso Full), con l'obiettivo di aiutare gli studenti a trovare lavoro.\n\n4. **Digital Marketing/AI Specialist**: Questo corso insegna a impos

In [11]:
from langchain import hub

llm = ChatOpenAI(model="gpt-5",
                 api_key=os.getenv("openai_api_key"))

prompt = hub.pull("rlm/rag-prompt")

chain = prompt | llm | StrOutputParser()

chain.invoke({
    "context": knowledge_base,
    "question": question
})

'Sì: Develhope forma profili pronti all’inserimento (Back-end Java, Full‑Stack, Digital Marketing/AI, Data & AI Analyst) e oltre 300 aziende assumono i loro studenti. Puoi attingere a questo bacino di talenti per le tue posizioni aperte. Per iniziare, compila il form e prenota una call con il team per illustrare le tue esigenze di recruiting.'

In [13]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""You are an assistant for Develhope for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.

Question: {question} 

Context: {context} 

Answer:""")

chain = prompt | llm | StrOutputParser()

chain.invoke({
    "context": knowledge_base,
    "question": question
})

'Sì. Develhope forma profili pronti all’inserimento (Backend Java, Full-Stack, Digital Marketing/AI, Data & AI Analyst) e collabora con oltre 300 aziende che assumono i nostri studenti; possiamo metterti in contatto con candidati idonei. Per iniziare, compila il form sul sito e fissa una call con il team Develhope.'